<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e6_make_errors_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# !pip install kiwipiepy
# # !pip install nltk
# # !pip install
# # !pip install selenium

# #코랩에서 구글드라이버 연동
# # !apt-get update
# # !apt install chromium-chromedriver

In [31]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 0.Data Load

In [32]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [33]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280765 entries, 0 to 585940
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  280765 non-null  object
 1   lawName   280765 non-null  object
 2   ENG       280765 non-null  object
 3   KOR       280765 non-null  object
dtypes: object(4)
memory usage: 10.7+ MB


# 1.Data Fiiltering

In [35]:
keyword_idx = df.loc[df.lawName.str.contains('은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대')|df.category.str.contains('경제|금융|상업')].index

In [36]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [37]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 45093	통화/국채/금융: 16004
키워드-통국금: 29089
통국금-키워드: 0
전체:  45093


In [38]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
# df_samp = df.copy(deep=True)

In [39]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
# df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
df_samp["errLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

In [40]:
df_samp.head()

,category,lawName,ENG,KOR,curHyp,errors,errLog
547115,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where it is impracticable to identify the loca...,"사업 또는 사업장(이하 ""사업""이라 한다)의 이전 등으로 사업의 소재지를 파악하기 ...","사업 또는 사업장(이하 ""사업""이라 한다)의 이전 등으로 사업의 소재지를 파악하기 ...",[],{}
547116,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,For full-time employees who receive a fixed am...,통상근로자로서 월정액으로 보수를 지급받는 근로자에게는 월단위 기준보수를 적용한다.,통상근로자로서 월정액으로 보수를 지급받는 근로자에게는 월단위 기준보수를 적용한다.,[],{}
547118,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard monthly remuneration shall apply ...,"「고용보험법」 제77조의2제1항에 따른 예술인(이하 ""예술인""이라 한다)에게는 월단...","「고용보험법」 제77조의2제1항에 따른 예술인(이하 ""예술인""이라 한다)에게는 월단...",[],{}
547119,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard monthly remuneration shall apply ...,"「고용보험법」 제77조의6제1항에 따른 노무제공자(이하 ""노무제공자""라 한다)에게는...","「고용보험법」 제77조의6제1항에 따른 노무제공자(이하 ""노무제공자""라 한다)에게는...",[],{}
547128,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Except as otherwise expressly provided for in ...,이 영에서 규정된 사업의 범위에 관하여 이 영에 특별한 규정이 없으면 「통계법」 제...,이 영에서 규정된 사업의 범위에 관하여 이 영에 특별한 규정이 없으면 「통계법」 제...,[],{}


**주요 함수**

In [128]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["errLog"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "errLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def dup_drop():
    for k in dindex_dict.keys():
        dindex_dict[k] = list(set(dindex_dict[k]))

def check_idx(df):
    dup_drop()
    for k in dindex_dict.keys():
        print(f"\t  {k}: {len(dindex_dict[k])}", end='\t')
        print(df.loc[df.errLog.apply(lambda x: k in x.keys())].shape[0])

def check_memorize(data):
    df = copy_df(data.loc[:, ["curHyp","errLog"]], col=["errLog"])
    result_list = []
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        d = df.loc[i, "errLog"]
        result_dict = dict()
        for k in d.keys():
            start = d[k]["start_idx"]
            end = start+len(d[k]["text"])
            result_dict[k] = cur_hyp[start:end]
        result_list.append(result_dict)
    df["validation"] = result_list
    return df

def move_index(df, idx, start, word, conv_word=None):
    err_log = copy.deepcopy(df.loc[idx, "errLog"])
    loop_keys = err_log.keys()
    for k in loop_keys:
        pre_idx = err_log[k]["start_idx"]
        if pre_idx > start:
            if conv_word==None:
                df.loc[idx, "errLog"][k]["start_idx"] = pre_idx + (len(word)+1)
            else:
                if len(conv_word) >= len(word):
                    df.loc[idx, "errLog"][k]["start_idx"] = pre_idx+len(conv_word)-len(word)
                else:
                    df.loc[idx, "errLog"][k]["start_idx"] = pre_idx-(len(word)-len(conv_word))

def replace_inteli(df, idx, keyword, rword):
    errLog = df.loc[idx ,"errLog"]
    cur_hyp = df.loc[idx, "curHyp"]

    text_dict=dict()
    for k in errLog.keys():
        if k=="omission": continue
        text, sindex = errLog[k]["text"], errLog[k]["start_idx"]
        text_dict[text] = (sindex, sindex+len(text))

    reject_range = []
    for key in text_dict.keys():
        if keyword in key:
            reject_range.append(text_dict[key])
    try:
        for f in re.finditer(keyword, cur_hyp):
            start, end = f.start(), f.end()
            is_reject = False
            for r_s, r_e in reject_range:
                if r_s <= start < r_e:
                    is_reject=True
                elif start <= r_s < r_e <= end:
                    is_rejcet=True

            if is_reject: continue
            while start>=0 and cur_hyp[start]!=" ":
                start-=1
            while end<len(cur_hyp) and cur_hyp[end]!=" ":
                end+=1
            start += 1

            word = cur_hyp[start:end]

            if word in text_dict.keys():
                start_idx = text_dict[word]
                if start==start_idx: continue
                break
            break
    except:
        raise print(f"{keyword} {rword}\n{cur_hyp}")

    try:
        add_word = word.replace(keyword, rword)
        conv_sentence = cur_hyp[:start]+add_word+cur_hyp[start+len(word):]
        return conv_sentence, start, add_word, word
    except:
        return None

def can_noise(df, i):
    cur_hyp = df.loc[i, "curHyp"]
    errLog = df.loc[i, "errLog"]
    for k in errLog.keys():
        if k=="omission": continue
        start, text = errLog[k]["start_idx"], errLog[k]["text"]
        end = start+len(text)
        cur_hyp = cur_hyp[:start]+("ㅱ"*len(text))+cur_hyp[end:]
    return cur_hyp

def errors_report(df, rate=False):
    sevs = {"critical":0, "major":1, "minor":2, "neutral":3}
    err_types = {
        'omission':[0, 0, 0, 0],
        'addition':[0, 0, 0, 0],
        'mis-translation':[0, 0, 0, 0],
        'over-translation':[0, 0, 0, 0],
        'under-translation':[0, 0, 0, 0],
        'grammer':[0, 0, 0, 0]
        }
    for i in df.index:
        errors = df.loc[i, "errors"]
        for d in errors:
            err_types[d["type"]][sevs[d["severity"]]]+=1
    report_df = pd.DataFrame(err_types, index=sevs.keys())
    report_df["sum"] = report_df.apply(np.sum, axis=1)
    report_df.loc["sum"] = report_df.apply(np.sum, axis=0)
    if rate:
        return_df = pd.DataFrame()
        for col in report_df.columns:
            return_df[col] = report_df[col].apply(lambda x: round((100*x)/report_df.loc["sum", "sum"], 2))
        return return_df
    else:
        return report_df

def find_errors(df, dindex_dict):
    temp = check_memorize(df)
    result = dict()
    for errType in dindex_dict.keys():
        if key=="omission": continue
        ttp = temp.loc[temp.errLog.apply(lambda i: errType in i)]
        result[errType] = ttp.loc[ttp["errLog"].apply(lambda x: x[errType]["text"]) != ttp["validation"].apply(lambda y: y[errType])]
    return result

# 2.품사 태깅

In [42]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547115    [(사업, NNG, 0, 2), (또는, MAJ, 3, 2), (사업장, NNG, ...
547116    [(통상, NNG, 0, 2), (근로자, NNG, 2, 3), (로서, JKB, ...
547118    [(「, SSO, 0, 1), (고용보험법, NNP, 1, 5), (」, SSC, ...
547119    [(「, SSO, 0, 1), (고용보험법, NNP, 1, 5), (」, SSC, ...
547128    [(이, MM, 0, 1), (영, NNG, 2, 1), (에서, JKB, 3, 2...
                                ...                        
521845    [(이, MM, 0, 1), (법, NNG, 2, 1), (시행, NNG, 4, 2...
521846    [(장기, NNG, 0, 2), (요양, NNG, 2, 2), (재, XPN, 4,...
521847    [(이, MM, 0, 1), (법, NNG, 2, 1), (시행, NNG, 4, 2...
521848    [(장기, NNG, 0, 2), (요양, NNG, 2, 2), (기관, NNG, 4...
521849    [(장기, NNG, 0, 2), (요양, NNG, 2, 2), (기관, NNG, 4...
Name: KOR, Length: 45092, dtype: object

In [43]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547115    {'NNG': ['사업', '사업장', '이하', '사업', '이전', '사업', ...
547116    {'NNG': ['통상', '근로자', '월', '정액', '보수', '지급', '...
547118    {'SSO': ['「', '(', '"'], 'NNP': ['고용보험법'], 'SS...
547119    {'SSO': ['「', '(', '"'], 'NNP': ['고용보험법'], 'SS...
547128    {'MM': ['이', '이'], 'NNG': ['영', '규정', '사업', '범...
                                ...                        
521845    {'MM': ['이'], 'NNG': ['법', '시행', '후', '위원', '임...
521846    {'NNG': ['장기', '요양', '심사', '위원회', '위원', '구성', ...
521847    {'MM': ['이'], 'NNG': ['법', '시행', '당시', '노인', '...
521848    {'NNG': ['장기', '요양', '기관', '지정', '목적', '법', '시...
521849    {'NNG': ['장기', '요양', '기관', '지정', '목적', '노인', '...
Name: KOR, Length: 45092, dtype: object

In [ ]:
# nng_dict = dict()
# for tokens in count_tokens.apply(lambda x: x["NNG"]):
#     for t in tokens:
#         try:
#             nng_dict[t] += 1
#         except:
#             nng_dict[t] = 1
# len(nng_dict.keys())

5261

In [45]:
from tqdm.notebook import tqdm

nouns_set = set()
for tokens in tqdm(count_tokens):
    for key in tokens.keys():
        if key=="NNG":
            nouns_set = nouns_set.union(set(tokens[key]))
            break
nouns = list(nouns_set)

  0%|          | 0/45092 [00:00<?, ?it/s]

# 3.Error 생성

In [46]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}

## Omission

### Omission - Critical

In [47]:
cr_om_index = random_index(df_samp.curHyp, 0.02, 10)
# df_samp.loc[cr_om_index]

In [49]:
# -- Omission : Critical
def critical_omission(df, ratio, seed):
    df_t = copy_df(df)
    cr_om_index = random_index(df_t.curHyp, ratio, seed)
    dindex_dict["omission"]+=list(cr_om_index)
    result = df_t.loc[cr_om_index]
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(seed)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in cr_om_index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")
        result.loc[i, "errLog"]["omission"] = {"start_idx": sidx, "text": reduce_sentence}

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return convert_df(df, result)

df_crom = critical_omission(df_samp, 0.02, 10)

In [50]:
df_crom.loc[cr_om_index].shape

(901, 7)

In [ ]:
# df_crom.loc[cr_om_index]

In [51]:
check_idx(df_crom)

	  omission: 901	901
	  addition: 0	0
	  mis-translation: 0	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


### Omission - Major

In [53]:
tag = "NNP"
np.random.seed(20)
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
np.random.seed(20)
change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
chv_index = random_index(ch6, 0.65, 20)

In [56]:
# -- Omission : Major
def major_omission(df_local, ratio, seed, tag, slen, kiwi_tokens):
    df_t = copy_df(df_local)
    np.random.seed(seed)
    nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
    np.random.seed(seed)
    change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
    ch6 = change_val[change_val.apply(lambda x: len(x[0])) > slen]
    chv_index = random_index(ch6, ratio, seed)
    # drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
    ch_srs = ch6.loc[chv_index]
    result = df_t.loc[chv_index]

    for i in chv_index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            dindex_dict["omission"]+=[i]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
            result.loc[i, "errLog"]["omission"] = {"start_idx": start, "text": del_unit}
        except:
            pass
    return convert_df(df_local, result)

df_maom = major_omission(df_crom, 0.65, 20, "NNP", 3, kiwi_tokens)
# df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [57]:
df_maom.loc[df_maom.errors.apply(len)==1].shape[0]

3977

In [ ]:
# df_maom.loc[df_maom.errors.apply(len)==1]

In [58]:
check_idx(df_maom)

	  omission: 3977	3977
	  addition: 0	0
	  mis-translation: 0	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


In [59]:
df_mo = major_omission(df_maom, 0.55, 36, "NNG", 2, kiwi_tokens)

In [60]:
check_idx(df_mo)

	  omission: 6359	6359
	  addition: 0	0
	  mis-translation: 0	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


In [ ]:
# df_mo.loc[df_mo.errors.apply(len)==1]

### Omission - Neutral

In [61]:
sp_list = [".", ";", ":"]
np.random.shuffle(sp_list)
sp_list

['.', ':', ';']

In [62]:
test = "안녕하세요; 반갑습니다."
a = test.find(".")
test[:a] + test[a+1:]

'안녕하세요; 반갑습니다'

In [63]:
def omission_nt(df, ratio, seed):
    df_t = copy_df(df)
    np.random.seed(seed)
    on_index = random_index(df_t.drop(index=dindex_dict["omission"]), ratio, seed)
    on_index = df_t.loc[on_index][df_t.loc[on_index].curHyp.str.contains(r"\.|\;|\:")].index
    result = df_t.loc[on_index]

    sp_list = [".", ";", ":"]
    for i in on_index:
        np.random.shuffle(sp_list)
        for s in sp_list:
            cur_hyp = result.loc[i, "curHyp"]
            s_loc = cur_hyp.find(s)
            if s_loc==-1:
                continue
            break
        conv_hyp = cur_hyp[:s_loc]+cur_hyp[s_loc+1:]
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "neutral", s_loc, s, "omission")
        result.loc[i, "errLog"]["omission"] = {"start_idx": s_loc, "text": s}
        dindex_dict["omission"].append(i)

    return convert_df(df, result)

df_on = omission_nt(df_mo, 0.01, 111)

In [ ]:
# df_on.loc[df_on.errors.apply(lambda x: x[0]["severity"]=="neutral" if len(x)!=0 else False)]

In [64]:
check_idx(df_on)

	  omission: 6686	6686
	  addition: 0	0
	  mis-translation: 0	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


## Addition

In [180]:
dindex_dict["addition"] = []

###Addition-Major

In [181]:
ma_ad_index = random_index(df_on, 0.22, 55)
nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
nnpg_info.drop(index=drop_index, axis=0, inplace=True)
maad_index = list(set(ma_ad_index)-set(drop_index))

In [182]:
## Addition : Major
def major_addition(df, ratio, seed, count_tokens, nouns):
    np.random.seed(seed)
    df_t=copy_df(df)

    ma_ad_index = random_index(df_t, ratio, seed)
    nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
    nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
    drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
    nnpg_info.drop(index=drop_index, axis=0, inplace=True)
    ma_ad_index = list(set(ma_ad_index)-set(drop_index))
    np.random.seed(seed)
    add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
    np.random.seed(seed)
    add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
    result = df_t.loc[ma_ad_index]


    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in ma_ad_index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        if start < 0:
            conv_hyp = f"{add_word}"+" "+cur_hyp[start+1:].strip()
        else:
            conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        move_index(result, i, start+1, add_word)
        result.loc[i, "errLog"]["addition"] = {"start_idx": (start+1), "text": add_word}
        dindex_dict["addition"].append(i)

    return convert_df(df, result)

In [183]:
# temp = check_memorize(df_maad.loc[maad_index])
# ttp = temp.loc[temp.errLog.apply(lambda i: "addition" in i)]
# ttp.loc[ttp["errLog"].apply(lambda x: x["addition"]["text"]) != ttp["validation"].apply(lambda y: y["addition"])]

In [184]:
df_maad = major_addition(df_on, 0.22, 55, count_tokens, nouns)
check_idx(df_maad)

	  omission: 6686	6686
	  addition: 9718	9718
	  mis-translation: 2763	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


### Addition-Minor

In [185]:
amindex = random_index(df_maad.drop(index=dindex_dict["addition"]), 0.2, 234)

In [186]:
## -- Addition : Minor
def ma_iter(df, ratio, seed):
    df_t = copy_df(df)
    np.random.seed(seed)
    am_index = random_index(df_t.drop(index=dindex_dict["addition"]), ratio, seed)
    result = df_t.loc[am_index]

    for i in am_index:
        cur_hyp = result.loc[i, "curHyp"]
        sch = cur_hyp.split(" ")
        sch_idx = [i for i in range(len(sch))]
        np.random.shuffle(sch_idx)
        is_possible = False
        for rand_idx in sch_idx:
            random_dup = sch[rand_idx]
            conv_hyp=" ".join(sch[:rand_idx] + [random_dup] +sch[rand_idx:])
            if rand_idx==0:
                start = 0
            else:
                start = len(" ".join(sch[:rand_idx]))+1
            sps_dup = re.sub(r"[^ㄱ-ㅎ가-힣]", "", random_dup)
            if len(sps_dup) == len(random_dup):
                is_possible=True
                break
        if not is_possible:
            continue
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "minor", start, random_dup, "addition")

        move_index(result, i, start, random_dup)
        result.loc[i, "errLog"]["addition"] = {"start_idx": start, "text": random_dup}
        dindex_dict["addition"].append(i)

    return convert_df(df, result)

In [187]:
df_miter = ma_iter(df_maad, 0.2, 234)
check_idx(df_miter)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 2763	0
	  over-translation: 0	0
	  under-translation: 0	0
	  grammer: 0	0


In [80]:
# df_miter.loc[df_miter.errLog.apply(lambda x: "omission" in x.keys()) & df_miter.errLog.apply(lambda x: "addition" in x.keys())]

## Mis-Translation

In [270]:
dindex_dict["mis-translation"] = []

### MT - Minor

In [ ]:
te_drop = df_miter.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
transHyp

547115    사업 또는 사업장(이하 "사업"이라 한다)의 ㅱㅱㅱ 이전 등으로 사업의 소재지를 파...
547116    통상근로자로서 월정액으로 ㅱㅱㅱ 보수를 지급받는 근로자에게는 월단위 기준보수를 적용한다.
547118    「고용보험법」 제77조의2제1항에 따른 예술인(이하 "예술인"이라 한다)에게는 월단...
547119    제77조의6제1항에 따른 노무제공자(이하 "노무제공자"라 한다)에게는 ㅱㅱㅱ 월단위...
547128    이 영에서 규정된 사업의 범위에 관하여 이 영에 특별한 규정이 없으면 「통계법」 제...
                                ...                        
521845    이 법 시행 후 ㅱㅱㅱ 위원을 임명 또는 위촉할 당시 제56조제3항 후단의 개정규정...
521846    장기요양재심사위원회의 위원 구성에 관하여는 제1항에 따라 제56조제3항 후단의 개정...
521847    이 법 시행 당시 「노인장기요양보험법 시행규칙」 제23조에 따라 장기요양기관 지정신...
521848    장기요양기관으로 지정받을 목적으로 이 법 시행 당시 「노인복지법」 제35조제2항에 ...
521849    장기요양기관으로 지정받을 목적으로 「노인복지법」에 따른 노인의료복지시설 또는 재가노...
Length: 45092, dtype: object

In [ ]:
te_drop = df_miter.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
temp = te_drop.loc[transHyp.str.contains(r'하여야 한다')]
can_idx = random_index(temp, 0.3, 789)

In [271]:
## Mis-Translation : Minor
def hv_mistrans(df, ratio, seed):
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.str.contains(r'하여야 한다')]
    can_index = random_index(temp, ratio, seed)
    result = te_drop.loc[can_index]
    ch_words = ['해야만 한다', '해야 할 것이다']

    for i in can_index:
        np.random.shuffle(ch_words)
        ch_word = ch_words[0]
        cur_hyp = result.loc[i, "curHyp"]
        start = cur_hyp.find('하여야 한다')
        conv_hyp = cur_hyp[:start] + ch_word + cur_hyp[start+6:]

        result.loc[i, 'curHyp'] = conv_hyp
        input_errors(result, i, 'minor', start, ch_word, 'mis-translation')

        move_index(result, i, start, "하여야 한다", ch_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": ch_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)


In [272]:
# temp = check_memorize(df_can.loc[can_idx])
# temp
# ttp = temp.loc[temp.errLog.apply(lambda i: "mis-translation" in i)]
# ttp.loc[ttp["errLog"].apply(lambda x: x["mis-translation"]["text"]) != ttp["validation"].apply(lambda y: y["mis-translation"])]

In [273]:
df_can = hv_mistrans(df_miter, 0.3, 789)
check_idx(df_can)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 2763	2763
	  over-translation: 17078	0
	  under-translation: 3944	0
	  grammer: 0	0


In [228]:
res = find_errors(df_can, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [274]:
def ddo_trans(df, ratio, seed, val):
    df_t = copy_df(df)
    conv_list = ["내지", "및", "또는"]
    conv_list.remove(val)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    ival = "\s"+val+"\s"
    ddo_idx = random_index(te_drop.loc[transHyp.str.contains(ival, regex=True)], ratio, seed)
    result = te_drop.loc[ddo_idx]


    for i in ddo_idx:
        np.random.shuffle(conv_list)
        conv_word = conv_list[0]

        conv_hyp, start, fconv_word, word = replace_inteli(result, i, val, conv_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, fconv_word, 'mis-translation')

        move_index(result, i, start, word, fconv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": fconv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

In [275]:
df_ddo = ddo_trans(df_can, 0.2, 555, "또는")

In [276]:
res = find_errors(df_ddo, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [277]:
df_nae = ddo_trans(df_ddo, 0.25, 444, "내지")

In [278]:
res = find_errors(df_nae, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [279]:
df_mit = ddo_trans(df_nae, 0.3, 333, "및")

In [280]:
res = find_errors(df_mit, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [281]:
check_idx(df_mit)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 7612	7612
	  over-translation: 17078	0
	  under-translation: 3944	0
	  grammer: 0	0


### MT - Major

In [ ]:
re.split(r"[^\d]+",'제16조의10제3항')

['', '16', '10', '3', '']

In [282]:
patterns = r"제[\d]+[\w]+[\d]+[\w]+\s"
compiler = re.compile(patterns)
te_drop = df_mit.drop(index=dindex_dict["mis-translation"])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns)], 0.35, 987)
result = te_drop.loc[std_idx]

for i in std_idx:
    cur_hyp = result.loc[i, "curHyp"]
    all_find = list(compiler.finditer(cur_hyp))
    np.random.shuffle(all_find)
    for f in all_find:
        sts = f.group().strip()
        sts_nsp = re.sub(r"[가-힣]+", " ", sts)
        sts_list = re.split(r"[^\d]+", sts_nsp)[1:-1]
        sts_add = re.split(r"[\d]+", sts)
        pre_shuffle = copy.deepcopy(sts_list)
        if len(list(set(pre_shuffle)))==1: continue
        while True:
            np.random.shuffle(pre_shuffle)
            if sts_list!=pre_shuffle:
                break
        conv_hyp = sts_add.pop(0)
        for a, b in zip(sts_add, pre_shuffle):
            conv_hyp+=(b+a)
        break
    print(cur_hyp, sts)
    print(replace_inteli(result, i, sts, conv_hyp))
    break


법 제47조의2제3항에서 "법원의 무죄 판결이 확정되는 등 대통령령으로 정하는 사유"란 다음 각 호의 어느 하나에 해당하는 사유를 말한다. 다만, 제2호 또는 제3호의 경우 불송치 또는 불기소를 받은 이후 해당 사건이 다시 수사 및 기소되어 법원의 판결에 따라 유죄가 확정된 경우는 제외한다. 제47조의2제3항에서
('법 제2조의3제47항에서 "법원의 무죄 판결이 확정되는 등 대통령령으로 정하는 사유"란 다음 각 호의 어느 하나에 해당하는 사유를 말한다. 다만, 제2호 또는 제3호의 경우 불송치 또는 불기소를 받은 이후 해당 사건이 다시 수사 및 기소되어 법원의 판결에 따라 유죄가 확정된 경우는 제외한다.', 2, '제2조의3제47항에서', '제47조의2제3항에서')


In [283]:
def replace_numbers(df, ratio, seed):
    df_t = copy_df(df)
    patterns = r"제[\d]+[\w]+[\d]+[\w]+\s"
    compiler = re.compile(patterns)
    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns)], ratio, seed)
    result = te_drop.loc[std_idx]

    for i in std_idx:
        cur_hyp = result.loc[i, "curHyp"]
        all_find = list(compiler.finditer(cur_hyp))
        np.random.shuffle(all_find)
        for f in all_find:
            sts = f.group().strip()
            sts_nsp = re.sub(r"[가-힣]+", " ", sts)
            sts_list = re.split(r"[^\d]+", sts_nsp)[1:-1]
            sts_add = re.split(r"[\d]+", sts)
            pre_shuffle = copy.deepcopy(sts_list)
            if len(list(set(pre_shuffle)))==1: continue
            while True:
                np.random.shuffle(pre_shuffle)
                if sts_list!=pre_shuffle:
                    break
            break
        chyp = sts_add.pop(0)
        for a, b in zip(sts_add, pre_shuffle):
            chyp+=(b+a)

        conv_hyp, start, conv_word, word = replace_inteli(result, i, sts, chyp)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'major', start, conv_word, 'mis-translation')

        move_index(result, i, start, word, conv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": conv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

In [284]:
df_rpnm = replace_numbers(df_mit, .35, 987)
res = find_errors(df_rpnm, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [285]:
check_idx(df_rpnm)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 12273	12273
	  over-translation: 17078	0
	  under-translation: 3944	0
	  grammer: 0	0


### MT-Neutral

In [240]:
df_rpnm.loc[403045, "curHyp"]

'제37조제1항 또는 제3항을 위반하여 신용공여 및 예금등을 받거나 가지급금을 받은 자'

In [286]:
rep_dict = {
    r'\"[^\"]+\"':"'", r"\'[^\']+\'":'"',
    r'「[^」]+」':'"'
    }

In [287]:
def change_morphus(df, ratio, seed, rep_dict):
    df_t = copy_df(df_rpnm)
    patterns = "|".join(rep_dict.keys())

    te_drop = df_t.drop(index=dindex_dict["mis-translation"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns, regex=True)], ratio, seed)

    result = te_drop.loc[std_idx]

    rpd_keys = copy.deepcopy(list(rep_dict.keys()))
    for e, i in enumerate(std_idx):
        cur_hyp = result.loc[i, "curHyp"]
        np.random.shuffle(rpd_keys)
        for pattern in rpd_keys:
            find_list = list(re.finditer(pattern, cur_hyp))
            if not find_list: continue
            break

        np.random.shuffle(find_list)
        f = find_list[0]

        word, start = f.group(), f.start()
        r_word = (rep_dict[pattern]+word[1:-1]+rep_dict[pattern]).strip()
        res = replace_inteli(result, i, word, r_word)
        if res!=None:
            conv_hyp, start, conv_word, word = res
        else:
            continue

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'neutral', start, conv_word, 'mis-translation')

        move_index(result, i, start, word, conv_word)
        result.loc[i, "errLog"]["mis-translation"] = {"start_idx": start, "text": conv_word}
        dindex_dict["mis-translation"].append(i)
    return convert_df(df, result)

In [288]:
df_cmp = change_morphus(df_rpnm, 0.025, 777, rep_dict)
res = find_errors(df_cmp, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [289]:
check_idx(df_cmp)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 12438	12438
	  over-translation: 17078	0
	  under-translation: 3944	0
	  grammer: 0	0


In [ ]:
errors_report(df_cmp, True)

,omission,addition,mis-translation,over-translation,under-translation,grammer,sum
critical,2.51,0.00,0.00,0.0,0.0,0.0,2.51
major,15.19,27.06,13.00,0.0,0.0,0.0,55.25
minor,0.00,19.68,21.19,0.0,0.0,0.0,40.88
neutral,0.91,0.00,0.46,0.0,0.0,0.0,1.37
sum,18.61,46.74,34.65,0.0,0.0,0.0,100.00


In [ ]:
rep_dict={r"\s때":" 경우", r"\s경우":" 때"}
ratio, seed = 0.3, 908
errType = "mis-translation"

df_t = copy_df(df_cmp)
patterns = "|".join(rep_dict.keys())

te_drop = df_t.drop(index=dindex_dict[errType])
transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns, regex=True)], ratio, seed)
len(std_idx)

4106

In [ ]:
list(rep_dict.keys())[np.random.choice(list(range(0, len(rep_dict.keys()))))]

'\\s때'

In [290]:
rep_dict={r"\s때":[" 경우"], r"\s경우":[" 때"]}
ratio, seed = 0.4, 908

def change_word(df, ratio, seed, rep_dict, severity, errType):
    df_t = copy_df(df)
    patterns = "|".join(rep_dict.keys())

    te_drop = df_t.drop(index=dindex_dict[errType])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    std_idx = random_index(te_drop.loc[transHyp.str.contains(patterns, regex=True)], ratio, seed)

    result = te_drop.loc[std_idx]

    rpd_keys = copy.deepcopy(list(rep_dict.keys()))
    for e, i in enumerate(std_idx):
        cur_hyp = result.loc[i, "curHyp"]
        np.random.shuffle(rpd_keys)
        is_complete=False
        for pattern in rpd_keys:
            i_word = pattern
            o_word = rep_dict[pattern][np.random.choice(list(range(0, len(rep_dict[pattern]))))]

            find_list = list(re.finditer(pattern, cur_hyp))
            if len(find_list)==0: continue
            break
        np.random.shuffle(find_list)
        f = find_list[0]
        start, init_word = f.start(), f.group()
        res = replace_inteli(result, i, i_word, o_word)
        if res != None:
            conv_hyp, start, conv_word, word = res
        else:
            continue
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, severity, start, conv_word, errType)

        move_index(result, i, start, word, conv_word)
        result.loc[i, "errLog"][errType] = {"start_idx": start, "text": conv_word}
        dindex_dict[errType].append(i)
    return convert_df(df, result)

In [291]:
df_ch1 = change_word(df_cmp, 0.4, 908, rep_dict, "minor", "mis-translation")
res = find_errors(df_ch1, dindex_dict)
res["addition"]

,curHyp,errLog,validation


In [247]:
check_idx(df_ch1)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 17878	17878
	  over-translation: 6998	0
	  under-translation: 3958	0
	  grammer: 0	0


## Over / Under Translation

In [292]:
dindex_dict['over-translation'] = []
dindex_dict['under-translation'] = []

In [249]:
import pickle
with open(os.path.join(gdrive_path, "etc_files/under_crawl.pkl"), "rb") as f:
    under = pickle.load(f)

with open(os.path.join(gdrive_path, "etc_files/over_crawl.pkl"), "rb") as f:
    over = pickle.load(f)

In [293]:
df_over = change_word(df_ch1, 0.2, 808, over, "major", "over-translation")
df_under = change_word(df_over, 0.2, 707, under, "major", "under-translation")

In [294]:
res = find_errors(df_over, dindex_dict)
print(res["addition"].shape[0], res["mis-translation"].shape[0])

0 0


In [295]:
res = find_errors(df_under, dindex_dict)
print(res["addition"].shape[0], res["mis-translation"].shape[0])

0 0


In [296]:
check_idx(df_under)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 17878	17878
	  over-translation: 6997	6997
	  under-translation: 3693	3693
	  grammer: 0	0


## Grammer

### Grammer-Minor

In [297]:
dindex_dict["grammer"] = []

In [300]:
def josa_grammer(df, ratio, seed):
    df_t = copy_df(df)
    js_mstr_index = random_index(df_t.drop(index=dindex_dict["grammer"]), ratio, seed)
    js_tokens = df_t.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))

    jsd = {
        "가":"이", "을":"를", "로써":"으로써", "로서":"으로서",
        "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나",
        "와":"과", "랑":"이랑", "이랑":"랑", "며":"이며", "이며":"며"
        }
    keys = list(jsd.keys())
    for key in keys:
        jsd[jsd[key]] = key

    tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
    need_josa = None
    for i, tag in enumerate(tags):
        if i==0:
            need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
        else:
            need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

    drop_index = need_josa[need_josa.apply(len)==0].index
    js_mstr_index = list(set(js_mstr_index)-set(drop_index))
    need_josa.drop(index=drop_index, axis=0, inplace=True)
    josa_elem = need_josa.apply(lambda x: list(set([val[0] for val in x])))
    result = df_t.loc[js_mstr_index]

    for i in js_mstr_index:
        cur_hyp = result.loc[i, "curHyp"]
        target = josa_elem[i]
        np.random.shuffle(target)
        for word in target:
            try:
                conv_hyp, start, conv_word, word = replace_inteli(result, i, word, jsd[word])
                result.loc[i, "curHyp"] = conv_hyp
                input_errors(result, i, 'minor', start, conv_word, 'grammer')

                move_index(result, i, start, word, conv_word)
                result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}
                dindex_dict["grammer"].append(i)
                break
            except:
                pass

    return convert_df(df, result)

In [301]:
df_jmt = josa_grammer(df_under, 0.2, 1010)
res = find_errors(df_jmt, dindex_dict)
for key in res.keys():
    print(f"{key}: {res[key].shape[0]}", end='\t')

omission: 6686	addition: 0	mis-translation: 0	over-translation: 0	under-translation: 0	grammer: 0	

In [302]:
check_idx(df_jmt)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 17878	17878
	  over-translation: 6997	6997
	  under-translation: 3693	3693
	  grammer: 8697	8697


In [305]:
def hada_gram(df, ratio, seed):
    df_t = copy_df(df)
    te_drop = df_t.drop(index=dindex_dict["grammer"])
    transHyp = pd.Series(te_drop.index.map(lambda x: can_noise(te_drop, x)), index=te_drop.index)
    temp = te_drop.loc[transHyp.apply(lambda x: "한다" in x)]
    handa_index = random_index(temp, ratio, seed)
    result = df_t.loc[handa_index]
    replace_list = ["하다", "함"]

    for i in handa_index:
        cur_hyp = result.loc[i, "curHyp"]
        r_word = np.random.choice(replace_list)
        conv_hyp, start, conv_word, word = replace_inteli(result, i, "한다", r_word)

        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, conv_word, 'grammer')

        move_index(result, i, start, word, conv_word)
        result.loc[i, "errLog"]["grammer"] = {"start_idx": start, "text": conv_word}
        dindex_dict["grammer"].append(i)
    return convert_df(df, result)

In [306]:
df_hada = hada_gram(df_jmt, 0.4, 141)
res = find_errors(df_hada, dindex_dict)
for key in res.keys():
    print(f"{key}: {res[key].shape[0]}", end='\t')

omission: 6686	addition: 0	mis-translation: 0	over-translation: 0	under-translation: 0	grammer: 0	

In [307]:
check_idx(df_hada)

	  omission: 6686	6686
	  addition: 16788	16788
	  mis-translation: 17878	17878
	  over-translation: 6997	6997
	  under-translation: 3693	3693
	  grammer: 17234	17234


In [308]:
df_hada.loc[df_hada.errors.apply(len)>0].shape[0]

38450

# 후처리

In [ ]:
def convert_index(df_new):
    df = copy_df(df_new)
    new_errs = []
    for i in df.index:
        errors, errLog = df.loc[i, "errors"], df.loc[i, "errLog"]

        for i, err in enumerate(df.loc[i, "errors"]):
            errors[i]['start_idx'] = errLog[err["type"]]["start_idx"]

        new_errs.append(errors)
    df["errors"] = new_errs
    return df
temp = convert_index(df_mit)

In [ ]:
# df_mit.loc[df_jmt.errLog.apply(lambda x: len(x)>2)].iloc[:10]

In [ ]:
# temp.loc[df_jmt.errLog.apply(lambda x: len(x)>2)].iloc[:10]

In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_can)

In [ ]:
with open(os.path.join(gdrive_path, "data_0825.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)